In [70]:
import random
import numpy as np

In [1]:
class RowVectorFloat:
    def __init__(self, elements):
        self.elements = list(elements)

    def __str__(self):
        return ' '.join(map(str, self.elements))

    def __len__(self):
        return len(self.elements)

    def __getitem__(self, index):
        return self.elements[index]

    def __setitem__(self, index, value):
        self.elements[index] = value

    def __add__(self, other):
        if len(self) != len(other):
            raise Exception("Vector dimensions must match for addition")
        result_elements = [x + y for x, y in zip(self.elements, other.elements)]
        return RowVectorFloat(result_elements)

    def __sub__(self, other):
        if len(self) != len(other):
            raise Exception("Vector dimensions must match for subtraction")
        result_elements = [x - y for x, y in zip(self.elements, other.elements)]
        return RowVectorFloat(result_elements)

    def __mul__(self, scalar):
        result_elements = [x * scalar for x in self.elements]
        return RowVectorFloat(result_elements)

    def __rmul__(self, scalar):
        return self.__mul__(scalar)

In [102]:
class SquareMatrixFloat:
    def __init__(self, n):
        self.matrix = [RowVectorFloat([0] * n) for _ in range(n)]
        self.size = n

    def __str__(self):
        s = "The matrix is: \n"
        s = s + '\n'.join(['\t'.join([f"{elem:.2f}" for elem in row.elements]) for row in self.matrix])
        return s
        # return '\n'.join(map(str, self.matrix))
    
    def __setitem__(self, index, value):
        self.matrix[index] = value

    def sampleSymmetric(self):
        for i in range(self.size):
            for j in range(i,self.size):
                self.matrix[i][j] = self.matrix[j][i] = random.random()
    
    def toRowEchelonForm(self):
        pivot = 0
        for i in range(self.size):
            # Find the pivot row
            pivot_row = None
            for j in range(i, self.size):
                if self.matrix[j][pivot] != 0:
                    pivot_row = j
                    break
            
            # Swap rows if necessary
            if pivot_row is not None:
                self.matrix[i], self.matrix[pivot_row] = self.matrix[pivot_row], self.matrix[i]
            else:
                continue
            
            # Scale the pivot row
            pivot_element = self.matrix[i][pivot]
            self.matrix[i] = self.matrix[i] * (1 / pivot_element)
            
            # Eliminate below the pivot
            for j in range(i + 1, self.size):
                factor = self.matrix[j][pivot]
                self.matrix[j] = self.matrix[j] - (self.matrix[i] * factor)
            
            pivot += 1
        
    def isDRDominant(self):
        for i in range(len(self.matrix)):
            diagonal_value = abs(self.matrix[i].elements[i])
            row_sum = sum(abs(x) for x in self.matrix[i].elements if x != diagonal_value)
            if row_sum >= diagonal_value:
                return False
        return True

    def jSolve(self, b, m):
        if not self.isDRDominant():
            raise Exception("Not solving because convergence is not guranteed.")

        A = np.array([[elem for elem in row.elements] for row in self.matrix])
        x = np.zeros(len(b))
        residuals = []

        for _ in range(m):
            x_new = np.zeros(len(b))
            for i in range(len(b)):
                s = np.dot(A[i], x) - A[i][i] * x[i]
                x_new[i] = (b[i] - s) / A[i][i]
            residuals.append(np.linalg.norm(np.dot(A, x_new) - b))
            x = x_new

        return x, residuals
    
    def gsSolve(self, b, m):
        if not self.isDRDominant():
            raise ValueError("Matrix is not diagonally row dominant")

        A = np.array([[elem for elem in row.elements] for row in self.matrix])
        x = np.zeros(len(b))
        residuals = []

        for _ in range(m):
            x_new = np.copy(x)
            for i in range(len(b)):
                s = np.dot(A[i], x_new) - A[i][i] * x_new[i]
                x_new[i] = (b[i] - s) / A[i][i]
            residuals.append(np.linalg.norm(np.dot(A, x_new) - b))
            x = x_new

        return x, residuals


In [103]:
s = SquareMatrixFloat(4)
s.sampleSymmetric()
(err, x) = s.gsSolve([1, 2, 3, 4], 10)
print(x)
print(err)

ValueError: Matrix is not diagonally row dominant